In [ ]:
# Bu hücre sadece gerekli kütüphaneleri kurar.
!pip install -q diffusers transformers accelerate torch torchvision Pillow
print("Kütüphaneler kuruldu veya zaten kurulu.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.4 MB/s eta 0:00:00
Kütüphaneler kuruldu veya zaten kurulu.


In [ ]:
!nvidia-smi

Sun May  4 10:35:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             47W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# Bu hücre kütüphaneleri import eder, Drive'ı bağlar, yolları ayarlar
# ve SADECE Stable Diffusion Inpainting modelini yükler.

import torch
from diffusers import StableDiffusionInpaintPipeline
from diffusers.utils import load_image
from PIL import Image
import os
from google.colab import drive
from IPython.display import display, Image as IPImage

# --- Google Drive Bağlantısı ---
try:
    drive.mount('/content/drive')
    drive_mounted = True
except Exception as e:
    print(f"!! Google Drive bağlanamadı: {e}. Dosya yolları çalışmayabilir.")
    drive_mounted = False

# GEREKLİYSE: resize_image_multiple_of_64 fonksiyonunu burada tanımla
# Eğer zaten başka bir hücrede tanımlıysa veya import edildiyse bu adıma gerek yok.

from PIL import Image
import math # round fonksiyonu için gerekebilir, ancak Python'da yerleşik

def resize_image_multiple_of_64(image, longest_side):
    """
    Bir görüntüyü, en uzun kenarı belirtilen 'longest_side' değerine eşit veya daha küçük olacak
    şekilde yeniden boyutlandırır, en boy oranını korur ve SONUÇ boyutlarının (genişlik ve yükseklik)
    64'ün katları olmasını sağlar.

    Args:
        image (PIL.Image.Image): Yeniden boyutlandırılacak PIL Görüntü nesnesi.
        longest_side (int): En uzun kenarın hedef maksimum piksel değeri.

    Returns:
        PIL.Image.Image: Yeniden boyutlandırılmış PIL Görüntü nesnesi.
        tuple: (final_w, final_h) son boyutları içeren tuple.
    """
    if not isinstance(image, Image.Image):
        raise TypeError("Girdi 'image' bir PIL Image nesnesi olmalıdır.")

    width, height = image.size
    original_size_str = f"{width}x{height}"

    # 1. Adım: Hedef boyuta göre ilk ölçeklemeyi hesapla
    if max(width, height) <= longest_side:
        target_w, target_h = width, height
        # print(f" - Görüntü zaten hedef 'longest_side' ({longest_side}) değerinden küçük veya eşit ({original_size_str}). 64'ün katlarına ayarlanacak.")
    else:
        if width > height:
            ratio = longest_side / width
            target_w = longest_side
            target_h = int(height * ratio)
        else:
            ratio = longest_side / height
            target_h = longest_side
            target_w = int(width * ratio)
        # print(f" - Görüntü {original_size_str} boyutundan 'longest_side'={longest_side} hedefine göre ~{target_w}x{target_h} boyutuna ölçekleniyor.")

    # 2. Adım: Hesaplanan hedef boyutları 64'ün en yakın katına yuvarla
    # round(x / 64.0) * 64 yerine math.floor veya math.ceil kullanmak daha deterministik olabilir,
    # ancak round genellikle işe yarar. Önemli olan tutarlılıktır.
    final_w = max(64, round(target_w / 64) * 64) # En az 64 olmasını sağla
    final_h = max(64, round(target_h / 64) * 64) # En az 64 olmasını sağla

    final_size_str = f"{final_w}x{final_h}"

    # 3. Adım: Eğer son boyutlar orijinal boyutlarla aynıysa, gereksiz yere yeniden boyutlandırma yapma
    if final_w == width and final_h == height:
        # print(f" - Son hedef boyut ({final_size_str}), orijinal boyutla aynı. Yeniden boyutlandırma atlanıyor.")
        return image, (width, height)

    # 4. Adım: Görüntüyü son, 64'ün katı olan boyutlara yeniden boyutlandır
    try:
        # print(f" - Görüntü son olarak {final_size_str} (64'ün katı) boyutuna yeniden boyutlandırılıyor...")
        resized_image = image.resize((final_w, final_h), Image.Resampling.LANCZOS)
        # print(f" - Yeniden boyutlandırma tamamlandı.")
        return resized_image, (final_w, final_h)
    except Exception as e:
        print(f"!! Görüntü yeniden boyutlandırma sırasında HATA (Orijinal: {original_size_str}, Hedef: {final_size_str}): {e}")
        raise # Hatayı yukarıya bildir


# --- Temel Ayarlar ---
setup_complete = False # Başlangıçta False, sonunda True olacak
if drive_mounted:
    # LÜTFEN BU YOLU KENDİ GOOGLE DRIVE KLASÖR YAPINIZA GÖRE GÜNCELLEYİN!
    base_project_dir = '/content/drive/MyDrive/Research_Project_ImageEditing/' # Örnek yol, kendinizinkini yazın
    print(f"Google Drive bağlandı. Proje kök dizini: {base_project_dir}")

    image_dir = os.path.join(base_project_dir, "Images")
    mask_dir = os.path.join(base_project_dir, "Masks") # Maske klasörünü de tanımlayalım
    metadata_path = os.path.join(base_project_dir, "metadata.json") # Metadata yolunu da ekleyelim

    # Çıktıları bu model için ayrı bir klasöre kaydedelim
    # (Bu yapı, daha sonra LowRes/HighRes gibi alt klasörler eklemeye de uygun)
    output_dir_sd = os.path.join(base_project_dir, "Outputs", "StableDiffusionInpainting")

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Gerekli klasörleri oluştur (varsa hata verme)
    os.makedirs(output_dir_sd, exist_ok=True)
    # Maske ve Image klasörlerinin varlığını kontrol etmek iyi bir pratik olabilir
    if not os.path.isdir(image_dir): print(f"!! UYARI: Görüntü klasörü bulunamadı: {image_dir}")
    if not os.path.isdir(mask_dir): print(f"!! UYARI: Maske klasörü bulunamadı: {mask_dir}")
    if not os.path.isfile(metadata_path): print(f"!! UYARI: Metadata dosyası bulunamadı: {metadata_path}")

    print(f"Kullanılan cihaz: {device}")
    print(f"Girdi Görüntü klasörü: {image_dir}")
    print(f"Girdi Maske klasörü: {mask_dir}")
    print(f"Metadata dosyası: {metadata_path}")
    print(f"Çıktı klasörü (SD Inpainting): {output_dir_sd}")

    # --- Model Ayarları (Stable Diffusion Inpainting) ---
    # 'Native' çözünürlük SD v1.5 Inpainting için 512'dir.
    # Araştırman farklı bir sonuç verirse veya farklı denemek istersen burayı güncelleyebilirsin.
    sd_target_resolution = 512
    print(f"\nStable Diffusion Hedef Çözünürlük (En Uzun Kenar): {sd_target_resolution}px")

    # Model Parametreleri (Ayarlanabilir - Başlangıç değerleri)
    # Bu değerleri daha sonraki analiz veya denemelerle iyileştirebilirsin.
    sd_inference_steps = 50  # SD için yaygın bir değer
    sd_guidance_scale = 7.5 # Prompt'un ne kadar etkili olacağı (IP2P text_guidance gibi)
    # Stable Diffusion Inpainting genellikle 'image_guidance' kullanmaz,
    # maskelenmiş alana odaklanır ve prompt'a göre doldurur.
    print(f"Model Parametreleri: steps={sd_inference_steps}, guidance_scale={sd_guidance_scale}")

    setup_complete = True # Ayarlar tamamlandı


else:
    print("!! Drive bağlanmadığı için devam edilemiyor.")
    setup_complete = False

# Son durumu kontrol etmek için küçük bir özet
if setup_complete:
    print("\n--- Kurulum Tamamlandı")
else:
    print("\n--- Kurulum Tamamlanamadı ---")

Mounted at /content/drive
Google Drive bağlandı. Proje kök dizini: /content/drive/MyDrive/SNU_Project_ImageEditing/
Kullanılan cihaz: cuda
Girdi Görüntü klasörü: /content/drive/MyDrive/SNU_Project_ImageEditing/Images
Girdi Maske klasörü: /content/drive/MyDrive/SNU_Project_ImageEditing/Masks
Metadata dosyası: /content/drive/MyDrive/SNU_Project_ImageEditing/metadata.json
Çıktı klasörü (SD Inpainting): /content/drive/MyDrive/SNU_Project_ImageEditing/Outputs/StableDiffusionInpainting

Stable Diffusion Hedef Çözünürlük (En Uzun Kenar): 512px
Model Parametreleri: steps=50, guidance_scale=7.5

--- Kurulum Tamamlandı


In [ ]:
 # --- Model Yükleme (Sadece Stable Diffusion Inpainting) ---
if setup_complete:
    print("\nStable Diffusion Inpainting modeli yükleniyor (ilk çalıştırmada zaman alabilir)...")
    try:
        pipe_sd_inpaint = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)

        print("- Stable Diffusion Inpainting modeli başarıyla yüklendi.")
        model_loaded = True
    except Exception as e:
        print(f"!! SD Inpainting yüklenirken HATA: {e}")
        print("!! Bellek hatası alıyorsanız, Colab Runtime'ı yeniden başlatıp SADECE BU notebook'u çalıştırmayı deneyin.")
        pipe_sd_inpaint = None
        model_loaded = False
else:
    print("!! Temel ayarlar tamamlanamadığı için model yüklenemiyor.")
    model_loaded = False


Stable Diffusion Inpainting modeli yükleniyor (ilk çalıştırmada zaman alabilir)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/914 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

- Stable Diffusion Inpainting modeli başarıyla yüklendi.


In [ ]:
# Bu hücre metadata.json dosyasını okur ve tüm girdileri Stable Diffusion Inpainting ile işler.

import json
import time
from PIL import Image
import os
import torch # torch.cuda.empty_cache() için

# Gerekli değişkenlerin önceki hücrelerden geldiğini varsayıyoruz:
# model_loaded, setup_complete, pipe_sd_inpaint, metadata_path, image_dir, mask_dir, output_dir_sd,
# sd_target_resolution, sd_inference_steps, sd_guidance_scale, device

if model_loaded and setup_complete:
    print("\n--- Stable Diffusion Inpainting Toplu İşlem Başlatılıyor ---")

    # Metadata dosyasını yükle
    try:
        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)
        print(f"- Metadata dosyası başarıyla okundu ({len(metadata)} girdi bulundu).")
    except Exception as e:
        print(f"!! Metadata dosyası okunamadı: {e}")
        metadata = [] # Hata durumunda boş liste ile devam et

    failed_entries = [] # Hata alan girdileri takip etmek için liste
    processed_count = 0
    total_entries = len(metadata)
    loop_start_time = time.time()

    for i, entry in enumerate(metadata):
        entry_id = entry.get("entry_id", f"unknown_entry_{i+1}")
        print(f"\n[{i+1}/{total_entries}] İşleniyor: {entry_id}")
        entry_start_time = time.time()

        try:
            # Gerekli bilgileri JSON'dan al
            input_rel_path = entry.get("input_image_path")
            mask_rel_path = entry.get("mask_path") # Null olabilir
            prompt = entry.get("prompt_sd_k")
            negative_prompt = entry.get("negative_prompt", "") # Yoksa boş string

            if not input_rel_path or not prompt:
                raise ValueError(f"JSON girdisinde 'input_image_path' ({input_rel_path}) veya 'prompt_sd_k' ({prompt}) eksik.")

            input_abs_path = os.path.join(base_project_dir ,input_rel_path)

            # Maske yolunu oluştur ve null kontrolü yap
            mask_abs_path = None
            if mask_rel_path:
                mask_abs_path = os.path.join(base_project_dir ,mask_rel_path)

            # Çıktı dosya adını ve yolunu oluştur (Modele ve çözünürlüğe özel)
            output_filename = f"{entry_id}_SD_{sd_target_resolution}px_output.png"
            output_abs_path = os.path.join(output_dir_sd, output_filename)

            # Eğer çıktı zaten varsa atla (isteğe bağlı)
            # if os.path.exists(output_abs_path):
            #     print(f" - Çıktı dosyası zaten var, atlanıyor: {output_filename}")
            #     processed_count += 1
            #     # Belleği temizlemeye yardımcı olmak için bazen atlanan adımda bile cache temizlenebilir
            #     # if device == 'cuda': torch.cuda.empty_cache()
            #     continue

            # 1. Girdi resmini yükle
            if not os.path.exists(input_abs_path):
                raise FileNotFoundError(f"Girdi resmi bulunamadı: {input_abs_path}")
            init_image = Image.open(input_abs_path).convert("RGB")
            print(f" - Girdi resmi yüklendi: {input_rel_path} (Orijinal Boyut: {init_image.size})")

            # 2. Görüntüyü hedef çözünürlüğe indir (64'ün katı)
            resized_image, final_dims = resize_image_multiple_of_64(init_image, sd_target_resolution)
            print(f" - Girdi resmi yeniden boyutlandırıldı: {final_dims}")

            # 3. Maskeyi yükle veya oluştur
            if mask_abs_path and os.path.exists(mask_abs_path):
                mask_image_original = Image.open(mask_abs_path).convert("L") # Genellikle grayscale (L)
                print(f" - Maske dosyası yüklendi: {mask_rel_path} (Orijinal Boyut: {mask_image_original.size})")
                # Maskeyi de AYNI hedef boyuta yeniden boyutlandır
                resized_mask, mask_final_dims = resize_image_multiple_of_64(mask_image_original, sd_target_resolution)
                # Boyutların eşleştiğini doğrula (küçük farklar olabilir, ama 64 katı olmalı)
                if final_dims != mask_final_dims:
                     # Önemli: Eğer resize fonksiyonu deterministik değilse veya
                     # orijinal en boy oranları çok farklıysa boyutlar tam tutmayabilir.
                     # Bu durumda ya resize fonksiyonunu gözden geçir ya da maskeyi
                     # yeniden boyutlandırılmış ana görüntünün boyutlarına göre resize et:
                     print(f" - UYARI: Yeniden boyutlandırılmış resim ({final_dims}) ve maske ({mask_final_dims}) boyutları farklı! Maske, resim boyutuna zorlanıyor.")
                     resized_mask = resized_mask.resize(final_dims, Image.Resampling.NEAREST) # Maskeler için NEAREST daha iyi olabilir
                     print(f" - Maske son boyuta ({final_dims}) yeniden boyutlandırıldı (NEAREST).")
                else:
                     print(f" - Maske yeniden boyutlandırıldı: {mask_final_dims}")
            elif mask_rel_path is None:
                # mask_path null ise, global değişiklik - beyaz maske oluştur
                print(f" - 'mask_path' null. Global değişiklik için beyaz maske ({final_dims}) oluşturuluyor.")
                # Beyaz maskeyi yeniden boyutlandırılmış resmin boyutunda oluştur
                resized_mask = Image.new("L", final_dims, 255) # Beyaz = 255
            else:
                # mask_path var ama dosya yoksa hata ver
                 raise FileNotFoundError(f"Maske dosyası belirtilmiş ancak bulunamadı: {mask_abs_path}")

            # 4. Modeli çalıştır
            print(f" - Model çalıştırılıyor (Prompt: '{prompt[:60]}...')")
            model_run_start_time = time.time()

            # Pipeline'a resize edilmiş image ve maskeyi ver
            # Görüntüleri tensöre dönüştürmeye gerek yok, pipeline hallediyor.
            result = pipe_sd_inpaint(
                prompt=prompt,
                image=resized_image,
                mask_image=resized_mask,
                negative_prompt=negative_prompt if negative_prompt else None, # Boşsa None gönder
                num_inference_steps=sd_inference_steps,
                guidance_scale=sd_guidance_scale,
                width=final_dims[0], # Boyutları pipeline'a bildirmek bazen performansı artırabilir
                height=final_dims[1]
            )
            edited_image = result.images[0]

            model_run_end_time = time.time()
            print(f" - Model çalışması tamamlandı ({model_run_end_time - model_run_start_time:.2f} saniye).")

            # 5. Çıktıyı kaydet
            edited_image.save(output_abs_path)
            print(f" - Çıktı kaydedildi: {output_filename}")
            processed_count += 1

            # (İsteğe bağlı) Her adımdan sonra Colab'da gösterme
            # from IPython.display import display
            # display(edited_image.resize((resized_image.width // 2, resized_image.height // 2))) # Daha küçük göster

        except Exception as e:
            error_message = f"HATA ({entry_id}): {type(e).__name__} - {e}"
            import traceback # Detaylı hata için
            traceback.print_exc() # Hatanın tam izini yazdır
            print(f"!! {error_message}")
            failed_entries.append({"entry_id": entry_id, "error": error_message})

        finally:
            # Her döngü sonunda belleği biraz rahatlatmaya çalış (özellikle GPU'da)
            if device == 'cuda':
                torch.cuda.empty_cache()

        entry_end_time = time.time()
        print(f" - Girdi {entry_id} tamamlandı ({entry_end_time - entry_start_time:.2f} saniye).")
        # İsteğe bağlı: Çok uzun sürüyorsa veya bellek doluyorsa arada duraklatma
        # time.sleep(1)

    loop_end_time = time.time()
    print(f"\n--- Stable Diffusion Inpainting Toplu İşlem Tamamlandı ---")
    # print(f"Toplam Süre: {(loop_end_time - loop_start_t;ime)/60:.2f} dakika")
    print(f"Başarıyla İşlenen Girdi Sayısı: {processed_count}/{total_entries}")

    # Hataları Raporla
    if failed_entries:
        print("\n--- Hata Alan Girdiler ---")
        for fail in failed_entries:
            print(f"- ID: {fail['entry_id']}, Hata: {fail['error']}")
    else:
        print("\n- Tüm girdiler hatasız işlendi.")

else:
    print("!! Model yüklenemediği veya kurulum tamamlanmadığı için toplu işlem başlatılamıyor.")


--- Stable Diffusion Inpainting Toplu İşlem Başlatılıyor ---
- Metadata dosyası başarıyla okundu (96 girdi bulundu).

[1/96] İşleniyor: Portrait_01_WomanHatAdd
 - Girdi resmi yüklendi: Images/Portrait_01_WomanHatAdd.png (Orijinal Boyut: (3744, 5616))
 - Girdi resmi yeniden boyutlandırıldı: (320, 512)
 - Maske dosyası yüklendi: Masks/Portrait_01_WomanHatAdd_mask.png (Orijinal Boyut: (3744, 5616))
 - Maske yeniden boyutlandırıldı: (320, 512)
 - Model çalıştırılıyor (Prompt: 'Woman wearing a stylish summer hat, outdoor portrait...')


  0%|          | 0/50 [00:00<?, ?it/s]

 - Model çalışması tamamlandı (6.88 saniye).
 - Çıktı kaydedildi: Portrait_01_WomanHatAdd_SD_512px_output.png
 - Girdi Portrait_01_WomanHatAdd tamamlandı (9.65 saniye).

[2/96] İşleniyor: Portrait_02_ManNecklaceAdd
 - Girdi resmi yüklendi: Images/Portrait_02_ManNecklaceAdd.png (Orijinal Boyut: (3149, 4724))
 - Girdi resmi yeniden boyutlandırıldı: (320, 512)
 - Maske dosyası yüklendi: Masks/Portrait_02_ManNecklaceAdd_mask.png (Orijinal Boyut: (3149, 4724))
 - Maske yeniden boyutlandırıldı: (320, 512)
 - Model çalıştırılıyor (Prompt: 'Man wearing a silver chain necklace, t-shirt...')


  0%|          | 0/50 [00:00<?, ?it/s]

 - Model çalışması tamamlandı (5.56 saniye).
 - Çıktı kaydedildi: Portrait_02_ManNecklaceAdd_SD_512px_output.png
 - Girdi Portrait_02_ManNecklaceAdd tamamlandı (7.44 saniye).

[3/96] İşleniyor: Portrait_03_WomanGlassesRemove


...

 - Model çalışması tamamlandı (5.61 saniye).
 - Çıktı kaydedildi: Art_14_FantasySceneToEerie_SD_512px_output.png
 - Girdi Art_14_FantasySceneToEerie tamamlandı (7.26 saniye).

[95/96] İşleniyor: Art_15_MakeDrawingLookOld
 - Girdi resmi yüklendi: Images/Art_15_MakeDrawingLookOld.png (Orijinal Boyut: (3000, 3000))
 - Girdi resmi yeniden boyutlandırıldı: (512, 512)
 - 'mask_path' null. Global değişiklik için beyaz maske ((512, 512)) oluşturuluyor.
 - Model çalıştırılıyor (Prompt: 'Old, faded paper sketch texture applied to the drawing, yell...')


  0%|          | 0/50 [00:00<?, ?it/s]

 - Model çalışması tamamlandı (7.97 saniye).
 - Çıktı kaydedildi: Art_15_MakeDrawingLookOld_SD_512px_output.png
 - Girdi Art_15_MakeDrawingLookOld tamamlandı (8.88 saniye).

[96/96] İşleniyor: Art_16_AddDetailsToSketch
 - Girdi resmi yüklendi: Images/Art_16_AddDetailsToSketch.png (Orijinal Boyut: (3738, 2334))
 - Girdi resmi yeniden boyutlandırıldı: (512, 320)
 - 'mask_path' null. Global değişiklik için beyaz maske ((512, 320)) oluşturuluyor.
 - Model çalıştırılıyor (Prompt: 'Detailed sketch with intricate shading and texture, finished...')


  0%|          | 0/50 [00:00<?, ?it/s]

 - Model çalışması tamamlandı (5.83 saniye).
 - Çıktı kaydedildi: Art_16_AddDetailsToSketch_SD_512px_output.png
 - Girdi Art_16_AddDetailsToSketch tamamlandı (6.48 saniye).

--- Stable Diffusion Inpainting Toplu İşlem Tamamlandı ---
Başarıyla İşlenen Girdi Sayısı: 96/96

- Tüm girdiler hatasız işlendi.


In [ ]:
import json
import os
from PIL import Image
import torch
import time

# Bu kısımlar zaten mevcut koddan gelmeli
# model_loaded, setup_complete, pipe_sd_inpaint, metadata_path, output_dir_sd, sd_target_resolution,
# sd_inference_steps, sd_guidance_scale, device

# İşlem yapılacak yeni veriler
entries = [
    {
    "entry_id": "Portrait_11_ToComicStyle",
    "input_image_path": "Images/Portrait_11_ToComicStyle.png",
    "mask_path": None,
    "prompt_sd_k": "Portrait in the style of a comic book art, bold lines, vibrant colors",
    "negative_prompt": "photorealistic, realistic, photograph, blurry, low quality, 3D render"
  },
  {
    "entry_id": "Portrait_12_ToOilPainting",
    "input_image_path": "Images/Portrait_12_ToOilPainting.png",
    "mask_path": None,
    "prompt_sd_k": "Oil painting portrait, visible brushstrokes",
    "negative_prompt": "photorealistic, realistic, photograph, drawing, sketch, blurry, low quality"
  },
]

if model_loaded and setup_complete:
    print("\n--- Stable Diffusion Inpainting Tekrar Başlatılıyor ---")

    failed_entries = []  # Hata alan girdiler için liste
    processed_count = 0

    for entry in entries:
        entry_id = entry.get("entry_id", f"unknown_entry")
        print(f"\nİşleniyor: {entry_id}")

        try:
            # Gerekli bilgileri JSON'dan al
            input_rel_path = entry.get("input_image_path")
            mask_rel_path = entry.get("mask_path")  # Null olabilir
            prompt = entry.get("prompt_sd_k")
            negative_prompt = entry.get("negative_prompt", "")  # Yoksa boş string

            if not input_rel_path or not prompt:
                raise ValueError(f"JSON girdisinde 'input_image_path' ({input_rel_path}) veya 'prompt_sd_k' ({prompt}) eksik.")

            input_abs_path = os.path.join(base_project_dir, input_rel_path)

            # Maske yolunu oluştur ve null kontrolü yap
            mask_abs_path = None
            if mask_rel_path:
                mask_abs_path = os.path.join(base_project_dir, mask_rel_path)

            # Çıktı dosya adını ve yolunu oluştur (Modele ve çözünürlüğe özel)
            output_filename = f"{entry_id}_SD_{sd_target_resolution}px_output.png"
            output_abs_path = os.path.join(output_dir_sd, output_filename)

            # Girdi resmini yükle
            if not os.path.exists(input_abs_path):
                raise FileNotFoundError(f"Girdi resmi bulunamadı: {input_abs_path}")
            init_image = Image.open(input_abs_path).convert("RGB")
            print(f" - Girdi resmi yüklendi: {input_rel_path} (Orijinal Boyut: {init_image.size})")

            # Görüntüyü hedef çözünürlüğe indir (64'ün katı)
            resized_image, final_dims = resize_image_multiple_of_64(init_image, sd_target_resolution)
            print(f" - Girdi resmi yeniden boyutlandırıldı: {final_dims}")

            # Maske oluşturuluyor, mask_path null ise beyaz maske
            if mask_abs_path is None:
                print(f" - 'mask_path' null. Global değişiklik için beyaz maske ({final_dims}) oluşturuluyor.")
                resized_mask = Image.new("L", final_dims, 255)  # Beyaz = 255
            else:
                raise FileNotFoundError(f"Maske dosyası belirtilmiş ancak bulunamadı: {mask_abs_path}")

            # Modeli çalıştır
            print(f" - Model çalıştırılıyor (Prompt: '{prompt[:60]}...')")
            model_run_start_time = time.time()

            # Pipeline'a resize edilmiş image ve maskeyi ver
            result = pipe_sd_inpaint(
                prompt=prompt,
                image=resized_image,
                mask_image=resized_mask,
                negative_prompt=negative_prompt if negative_prompt else None,
                num_inference_steps=sd_inference_steps,
                guidance_scale=sd_guidance_scale,
                width=final_dims[0],
                height=final_dims[1]
            )
            edited_image = result.images[0]

            model_run_end_time = time.time()
            print(f" - Model çalışması tamamlandı ({model_run_end_time - model_run_start_time:.2f} saniye).")

            # Çıktıyı kaydet
            edited_image.save(output_abs_path)
            print(f" - Çıktı kaydedildi: {output_filename}")
            processed_count += 1

        except Exception as e:
            error_message = f"HATA ({entry_id}): {type(e).__name__} - {e}"
            import traceback  # Detaylı hata için
            traceback.print_exc()  # Hatanın tam izini yazdır
            print(f"!! {error_message}")
            failed_entries.append({"entry_id": entry_id, "error": error_message})

        # Belleği temizlemeye çalış (özellikle GPU'da)
        if device == 'cuda':
            torch.cuda.empty_cache()

    # Sonuçları yazdır
    print(f"\n--- Stable Diffusion Inpainting Tamamlandı ---")
    print(f"Başarıyla İşlenen Girdi Sayısı: {processed_count}/{len(entries)}")

    if failed_entries:
        print("\n--- Hata Alan Girdiler ---")
        for fail in failed_entries:
            print(f"- ID: {fail['entry_id']}, Hata: {fail['error']}")
    else:
        print("\n- Tüm girdiler hatasız işlendi.")
else:
    print("!! Model yüklenemediği veya kurulum tamamlanmadığı için işlem başlatılamıyor.")



--- Stable Diffusion Inpainting Tekrar Başlatılıyor ---

İşleniyor: Portrait_11_ToComicStyle
 - Girdi resmi yüklendi: Images/Portrait_11_ToComicStyle.png (Orijinal Boyut: (4912, 7360))
 - Girdi resmi yeniden boyutlandırıldı: (320, 512)
 - 'mask_path' null. Global değişiklik için beyaz maske ((320, 512)) oluşturuluyor.
 - Model çalıştırılıyor (Prompt: 'Portrait in the style of a comic book art, bold lines, vibra...')


  0%|          | 0/50 [00:00<?, ?it/s]

 - Model çalışması tamamlandı (3.51 saniye).
 - Çıktı kaydedildi: Portrait_11_ToComicStyle_SD_512px_output.png

İşleniyor: Portrait_12_ToOilPainting
 - Girdi resmi yüklendi: Images/Portrait_12_ToOilPainting.png (Orijinal Boyut: (3000, 4499))
 - Girdi resmi yeniden boyutlandırıldı: (320, 512)
 - 'mask_path' null. Global değişiklik için beyaz maske ((320, 512)) oluşturuluyor.
 - Model çalıştırılıyor (Prompt: 'Oil painting portrait, visible brushstrokes...')


  0%|          | 0/50 [00:00<?, ?it/s]

 - Model çalışması tamamlandı (2.37 saniye).
 - Çıktı kaydedildi: Portrait_12_ToOilPainting_SD_512px_output.png

--- Stable Diffusion Inpainting Tamamlandı ---
Başarıyla İşlenen Girdi Sayısı: 2/2

- Tüm girdiler hatasız işlendi.
